In [ ]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime, timedelta
import sys
sys.path.append('/Users/tarabaris/GitHub/odp-sdk-python/Examples')

## For SDK
from getpass import getpass
from odp_sdk import ODPClient

from odp_sdk.utils.numeric import *
from odp_sdk.utils.visual import *

## For plotting
import seaborn as sns
import matplotlib.pyplot as plt
import mpl_toolkits
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cmocean
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.lines import Line2D

## For geopandas
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon

## Extra functions
from tqdm import notebook
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_palette(sns.color_palette("hls", 47))

# Connect to API

In [ ]:
client = ODPClient(api_key=getpass(prompt='Insert your personal ODP API key:'), 
                       project="odp", client_name="odp")

Run function you will need to fetch polygons of different regions

In [ ]:
def get_poly(df):
    polys=[]
    for i in notebook.tqdm(range(len(df))):
        if df['poly_count'][i]>1:
            m_polys=[]
            for i in range(df['poly_count'][i]):
                df_p = client.sequences.data.retrieve(id=df['id'][i], start=1, end=None).to_pandas()
                m_polys.append(Polygon(zip(df_p[df_p.polygonId==i]['lat'], df_p[df_p.polygonId==i]['lon'])))
            polys.append(MultiPolygon(m_polys))
        else:
            polys.append(Polygon(zip(client.sequences.data.retrieve(id=df['id'][i], start=1, end=None).to_pandas()['lat'], client.sequences.data.retrieve(id=df['id'][i], start=1, end=None).to_pandas()['lon'])))
    df['geometry']=polys

# Get casts and data from Antarctic EEZ

First get all sequences of Marine Regions and EEZs that belong to Antarctica

In [ ]:
seqs = client.sequences.list(external_id_prefix = 'marine-regions-Intersect_EEZ_IHO', metadata={'SOVEREIGN1': 'ANTARCTICA'}, limit=-1).to_pandas()
seqs.head()

Create a dataframe with marine regions, eezs, and associated polygons

In [ ]:
df_ant = pd.DataFrame({'marine_regions':[seqs.iloc[i]['metadata']['MARREGION'] for i in range(len(seqs))], 
                      'MRGID':[seqs.iloc[i]['metadata']['MRGID'] for i in range(len(seqs))],
                      'IHO_Sea':[seqs.iloc[i]['metadata']['IHO_SEA'] for i in range(len(seqs))],
                      'EEZ':[seqs.iloc[i]['metadata']['EEZ'] for i in range(len(seqs))],
                      'SOVEREIGN1':[seqs.iloc[i]['metadata']['SOVEREIGN1'] for i in range(len(seqs))],
                      'id': seqs['id'],
                      'poly_count':[int(seqs.iloc[i]['metadata']['polygonCount']) for i in range(len(seqs))]})
get_poly(df_ant)
df_ant = gpd.GeoDataFrame(df_ant)

In [ ]:
df_ant.head()

Pull all casts from 2018 and find the ones that intersect with the Antarctic EEZ Polygons


In [ ]:
## Download casts from 2018 and turn into a geopandas dataframe
casts2018 = client.get_available_casts([-180, 180], [-90, 90], ['2018-01-01', '2018-12-31'], n_threads=35)
casts2018 = gpd.GeoDataFrame(
    casts2018, geometry=gpd.points_from_xy(casts2018.lon, casts2018.lat))
casts2018.head()

In [ ]:
## Find intersect of casts and Antarctic EEZ
casts2018_ant = gpd.sjoin(casts2018, df_ant, how="inner", op='intersects')
casts2018_ant.head(2)

You can then fetch the measurement data from each of these casts

In [ ]:
ant_data = client.download_data_from_casts(casts2018_ant.extId.unique(), n_threads=40)
ant_data.head()

And you can plot the measurements for Temperature in the Antarctic EEZ

In [ ]:
plot_casts('Temperature', ant_data, cmap=cmocean.cm.thermal)

# Assign World Seas Marine Regions to casts

In [ ]:
seqs = client.sequences.list(external_id_prefix = 'marine-regions-Intersect_EEZ_IHO', limit=-1).to_pandas()
seqs.head()

Create dataframe with name of marine region and polygons

In [ ]:
df_mr = pd.DataFrame({'marine_regions':[seqs.iloc[i]['metadata']['MARREGION'] for i in range(len(seqs))],
                      'iho_sea':[seqs.iloc[i]['metadata']['IHO_SEA'] for i in range(len(seqs))],
                      'MRGID':[seqs.iloc[i]['metadata']['MRGID'] for i in range(len(seqs))],
                      'poly_count':[int(seqs.iloc[i]['metadata']['polygonCount']) for i in range(len(seqs))],
                      'id':seqs['id']})
get_poly(df_mr)
df_mr = gpd.GeoDataFrame(df_mr)

In [ ]:
df_mr.head()

# Join oceanographic data to Marine Regions
Here we find the intersect of points from the casts with the polygons from the marine regions. 
The resulting dataframe will have the associated marine regions for each cast location. 

In [ ]:
casts2018_mr = gpd.sjoin(casts2018, df_mr, how="inner", op='intersects')

In [ ]:
casts2018_mr.head(2)

In [ ]:
def plot_marine_regions_data(df_casts, df_marine_regions, lat=[-90, 90], lon=[-180,180]):
    
    df_casts = df_casts[(df_casts.lat.between(lat[0], lat[1])) & (df_casts.lon.between(lon[0], lon[1]))]

    fig = plt.figure(figsize=(14, 14))
    colors = sns.color_palette('hls', n_colors=len(df_casts.marine_regions.unique()))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    mr = df_casts.marine_regions.unique()
    legend_elements = []
    for i, j in enumerate(mr):
        df_marine_regions[df_marine_regions.marine_regions == j]['geometry'].plot(ax=ax, markersize=5, color=colors[i], zorder=1);
        legend_elements.append(Line2D([0], [0], color = colors[i], lw=4, label=mr[i]))
    

    sns.scatterplot(x="lon", y="lat", data=df_casts, color = 'navy', s=20, marker='o', edgecolor='white', linewidths=0.05)
    ax.set_extent([lon[0], lon[1], lat[0], lat[1]],crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.7, linestyle=':')
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    ax.legend(handles=legend_elements, loc='lower center',
               ncol=3, borderaxespad=-12)

    geo_map(ax)

In [ ]:
plot_marine_regions_data(casts2018_mr, df_mr, lat=[-5,15], lon=[-100,-75])